In [0]:
!pip install -U sentence-transformers

In [0]:
import pandas as pd
import glob
from tqdm import tqdm
import re

In [0]:
#path = r'JSON_Files/' # use your path
all_files = glob.glob("*.json")

# df = pd.read_csv('data_searchAPI/output.csv')
li = []

for filename in all_files:
    df = pd.read_json(filename)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [0]:
frame.head()

In [0]:
selectColumns = ['title','siteName','pageUrl','text', 'date']
frame = frame[selectColumns]

frame.head()

In [0]:
frame = frame.rename(columns={"pageUrl": "url", "text": "snippet", "siteName": "domain"})

In [0]:
path = r'processed.csv' # use your path
# all_files = glob.glob(path + "/*.csv")

# df = pd.read_csv('data_searchAPI/output.csv')
# li = []

# for filename in all_files:
#    df = pd.read_csv(filename, index_col=None, header=0)
#    li.append(df)

# frame = pd.concat(li, axis=0, ignore_index=True)

# frame.head()

df_sv = pd.read_csv(path, dtype={
    'date': str, 
    'keyword': str
})

In [0]:
df_sv.head()

In [0]:
frame = [frame,df_sv]

In [0]:
frame_data= pd.concat(frame)

In [0]:
frame_data

In [0]:
print(frame_data.duplicated().sum())
print(frame_data.snippet.isnull().sum())

In [0]:
frame_data.drop_duplicates(keep='first')

In [0]:
frame_data['snippet'].fillna('no description', inplace=True)
frame_data['title'].fillna('no title', inplace=True)
frame_data.info()

In [0]:
frame_data = frame_data.drop(["date", "keyword"], axis=1)

In [0]:
frame_data['snippet'] = frame_data['snippet'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))

In [0]:
frame_data

In [0]:
df_dict = frame_data.to_dict()
len_text = len(df_dict["title"])
len_text

In [0]:
#keyword_list  = [] # keyword
snippet_description_list = [] # snippet_description
snippet_list = [] # snippet
title_list = [] # title
url_list = [] # url 
domain_list = [] # domain
for i in tqdm(range(0,len_text)):
  snippet = df_dict["snippet"][i].split("\n")
  snippet_description = df_dict["snippet"][i]
  title = df_dict["title"][i]
  url = df_dict["url"][i]
  domain = df_dict["domain"][i]
  for b in snippet:
    #keyword_list.append(keyword)
    snippet_list.append(b)
    snippet_description_list.append(snippet)
    title_list.append(title)
    url_list.append(url)
    domain_list.append(domain)

In [0]:
df_sentences = pd.DataFrame({"title":title_list},index=snippet_list)
df_sentences.to_csv("keyword.csv")
df_sentences.head()

In [0]:
df_sentences = pd.DataFrame({"title":title_list,"url":url_list,"domain":domain_list, "snippet_description": snippet_description_list },index=snippet_list)
df_sentences.to_csv("Processed_Full.csv")
df_sentences.head()
# "keyword":keyword_list,

In [0]:
df_sentences = pd.read_csv("keyword.csv")
df_sentences = df_sentences.set_index("Unnamed: 0")
df_sentences.head()

In [0]:
df_sentences = df_sentences["title"].to_dict()
df_sentences_list = list(df_sentences.keys())
len(df_sentences_list)

In [0]:
df_sentences_list = [str(d) for d in tqdm(df_sentences_list)]

In [0]:
df = pd.read_csv("Processed_Full.csv", index_col=0)
df.head()

In [0]:
#https://github.com/UKPLab/sentence-transformers/blob/master/examples/application_semantic_search.py
"""
This is a simple application for sentence embeddings: semantic search
We have a corpus with various sentences. Then, for a given query sentence,
we want to find the most similar sentence in this corpus.
This script outputs for various queries the top 5 most similar sentences in the corpus.
"""

from sentence_transformers import SentenceTransformer
import scipy.spatial
import pickle as pkl
embedder = SentenceTransformer('bert-base-nli-mean-tokens')

# Corpus with example sentences
corpus = df_sentences_list
corpus_embeddings = embedder.encode(corpus,show_progress_bar=True)
#with open("path/.pkl" , "rb") as file_:
#  corpus_embeddings = pd.load_pkl(file_)

# Query sentences:
queries = ['3d Printing technologies',
           'IOT devices that are new to 2020',
           'injection molding advances.']
query_embeddings = embedder.encode(queries,show_progress_bar=True)

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
closest_n = 10
print("\nTop 10 most similar sentences in corpus:")
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n=========================================================")
    print("==========================Query==============================")
    print("====",query,"=====")
    print("=========================================================")


    for idx, distance in results[0:closest_n]:
        print("Score:   ", "(Score: %.4f)" % (1-distance) , "\n" )
        print("Paragraph:   ", corpus[idx].strip(), "\n" )
        row_dict = df.loc[df.index== corpus[idx]].to_dict()
        # print("Keyword:  " , row_dict["keyword"][corpus[idx]] , "\n")
        print("Title:  " , row_dict["title"][corpus[idx]] , "\n")
        print("Domain:  " , row_dict["domain"][corpus[idx]] , "\n")
        print("Url:  " , row_dict["url"][corpus[idx]] , "\n")
        print("Snippet:  " , row_dict["snippet_description"][corpus[idx]] , "\n")
        print("-------------------------------------------")